In [ ]:
import pandas as pd
import numpy as np
import os
import time
import rtsvg

rt = rtsvg.RACETrack()

_base_dir_ = '../../../data/2014_vast/MC1/News Articles'

month_str_to_digits = {'January':'01', 'February':'02', 'March':'03',     'April':'04',   'May':'05',      'June':'06', 
                       'July':'07',    'August':'08',   'September':'09', 'October':'10', 'November':'11', 'December':'12'}
def normalizeDate(s):
    if s == '13June 2010':           s = '13 June 2010'
    if s == '21 January 2014  1405': s = '21 January 2014'
    while '  ' in s: s = s.replace('  ', ' ')
    for month_str in month_str_to_digits:
        if   s.startswith(month_str):
            _parts_ = s.split(' ')
            if len(_parts_) != 3: raise Exception(f'"{s}" does not have three parts')
            _month_str_ = _parts_[0]
            _month_     = month_str_to_digits[_month_str_]
            _day_str_   = _parts_[1]
            if _day_str_.endswith(','): _day_str_ = _day_str_[:-1]
            if len(_day_str_) == 1: _day_str_ = '0' + _day_str_
            _year_str_  = _parts_[2]
            if len(_year_str_) != 4: raise Exception(f'"{s}" year string is not four digits')
            return f'{_year_str_}-{_month_}-{_day_str_}'
        elif month_str in s:
            _parts_ = s.split(' ')
            if len(_parts_) == 3:
                _day_str_   = _parts_[0]
                if len(_day_str_) == 1: _day_str_ = '0' + _day_str_
                _month_str_ = _parts_[1]
                _month_     = month_str_to_digits[_month_str_]
                _year_str_  = _parts_[2]
                if len(_year_str_) != 4: raise Exception(f'"{s}" year string is not four digits')
                return f'{_year_str_}-{_month_}-{_day_str_}'
    return s.replace('/', '-')

def breakIntoSentences(_str_):
    _ret_, _tuples_ = [], rt.textExtractSentences(_str_)
    for _tuple_ in _tuples_: _ret_.append(_tuple_[0])
    return _ret_

def partLooksLikeADate(_part_):
    return len(_part_) <= 10 and _part_.count('/') == 2 # not perfect but upon inspection, results look correct

def separateArticle(_str_):
    found_locations = ['ABILA, Kronos', 'Abila, Kronos', 'CENTRUM, Tethys', 'DAVOS, Switzerland', 'ELODIS, Kronos']
    _source_, _title_, _published_, _location_, _author_, _sentences_ = None, None, None, None, None, []
    _parts_ = _str_.split('\n')
    for _part_i_ in range(len(_parts_)):
        _part_ = _parts_[_part_i_]
        if len(_part_) > 0 and _part_.startswith('<<') == False:
            if   _part_.startswith("b'SOURCE:"):   _source_    = _part_.replace("b'SOURCE:",  '')  .strip()
            elif _part_.startswith('SOURCE":'):    _source_    = _part_.replace('SOURCE":',   '')  .strip()
            elif _part_.startswith("TITLE:"):      
                _part_ = _part_.replace('TITLE:',     '')  .strip()
                if _part_ == "Maha Salo": _author_ = _part_
                else:                     _title_  = _part_
            elif _part_.startswith('AUTHOR:'):     _author_    = _part_.replace('AUTHOR:',    '')  .strip()
            elif _part_.startswith("PUBLISHED:"):
                _part_ = _part_.replace('PUBLISHED:', '').strip()
                if    _part_ == 'Petrus Gerhard':
                    _author_ = _part_
                elif  _part_ == 'By Haneson Ngohebo':
                    _author_ =  'Haneson Ngohebo'
                elif  _part_ == 'PROTESTS SWELL IN ABILA IN RESPONSE TO CALLS TO ACTION':
                    if _title_ is not None: print(f'Title already set to "{_title_}"')
                    _title_ = _part_
                else: _published_ = normalizeDate(_part_)
            elif _part_.startswith("LOCATION: TITLE:"): _title_    = _part_.replace('LOCATION: TITLE:',  '').strip() 
            elif _part_.startswith("LOCATION:"):
                 _maybe_location_ = _part_.replace('LOCATION:',         '').strip()
                 if _maybe_location_ in found_locations: _location_ = _maybe_location_
                 else: _sentences_.extend(_part_)
            elif partLooksLikeADate(_part_):
                _published_ = _part_.strip()
            else:                                  _sentences_.extend(breakIntoSentences(_part_))
    _elodis_location_ = 'ELODIS, KRONOS: '
    if _title_ is not None and _title_.startswith(_elodis_location_): _title_, _location_ = _title_[len(_elodis_location_):].strip(), _elodis_location_
    return _source_, _title_, _published_, _location_, _author_, _sentences_

_lu_       = {'directory':[], 'file':[], 'source':[], 'title':[], 'published':[], 'location':[], 'author':[], 'sentence':[], 'sentence_no':[]}
done       = set()

for _dir_ in os.listdir(_base_dir_):
    for _file_ in os.listdir(os.path.join(_base_dir_, _dir_)):
        if _file_ in done: continue
        _article_raw_ = open(os.path.join(_base_dir_, _dir_, _file_), 'rb').read()
        _src_, _title_, _published_, _location_, _author_, _sentences_ = separateArticle(str(_article_raw_).replace('\\r', '').replace('\\n', '\n'))
        if _src_ is None: _src_ = _dir_
        _sentence_no_ = 0
        for _ in _sentences_:
            if len(_) < 6: continue
            ts0_model = time.time()
            ts1_model = time.time()
            _lu_['directory'].append(_dir_)
            _lu_['file'].append(_file_)
            _lu_['source'].append(_src_)
            _lu_['title'].append(_title_)
            _lu_['published'].append(_published_)
            _lu_['location'].append(_location_)
            _lu_['author'].append(_author_)
            _lu_['sentence'].append(_)
            _lu_['sentence_no'].append(_sentence_no_)
            _sentence_no_ += 1
        # pd.DataFrame(_lu_).to_csv('llama3_8b_2014_vast_sbs.csv', index=False)
df = pd.DataFrame(_lu_).sort_values(['directory','file','sentence_no'])
df = rt.columnsAreTimestamps(df, 'published')
df.to_parquet('articles.parquet')
df.sample(10)

In [2]:
#import networkx as nx
#relates = [('directory','source')]
#g       = rt.createNetworkXGraph(df, relates)
#pos     = nx.spring_layout(g)
#igl     = rt.interactiveGraphPanel(df, ln_params={'relationships':relates, 'pos':pos, 'draw_labels':False, 'node_size':'small'}, w=800, h=600)
#igl